In [1]:
from tkinter import *
from tkinter import ttk
import tkinter.font as tkFont
from tkinter import font

In [2]:
from movie import Movie

In [3]:
from parse import parse_imdb_data

In [4]:
import os
import pickle

In [5]:
if os.path.isfile("movies.p"):
    movies = pickle.load(open("movies.p", "rb"))
else:
    movies = parse_imdb_data()
    pickle.dump(movies, open( "movies.p", "wb" ))

In [43]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>10000, m_))
#m_=list(filter(lambda m: m.mean_rating>8.5, m_))
len(m_)

8000

In [44]:
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) \
        for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    if(data[0][0].isnumeric()): # FIX: doesn't work for float
        data = [(float(d[0]), d[1]) for d in data]
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, \
        int(not descending)))

In [45]:
min_year = min_rating_count = min_mean_rating = float('inf')
max_year = max_rating_count = max_mean_rating = float('-inf')
for m in m_:
    min_year = min((min_year, m.year))
    max_year = max((max_year, m.year))
    min_rating_count = min((min_rating_count, m.num_ratings))
    max_rating_count = max((max_rating_count, m.num_ratings))
    min_mean_rating = min((min_mean_rating, m.mean_rating))
    max_mean_rating = max((max_mean_rating, m.mean_rating))    

In [46]:
if os.path.isfile("ratings.p"):
    ratings = pickle.load(open("ratings.p", "rb"))
else:
    ratings = {}

In [152]:
root = Tk()
root.title("IMDB MOVIE RECOMMENDER SYSTEM")

mainframe = ttk.Frame(root, padding="3 3 12 12")
mainframe.grid(column=0, row=0, sticky=(N, W, E, S))

filterframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Filter")
filterframe.grid(column=0, row=0, sticky=(N, W))
filterframe['borderwidth'] = 2
filterframe['relief'] = 'sunken'


yearfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Year")
yearfilterframe.grid(column=0, row=0, sticky=(N, E))
yearfilterframe['borderwidth'] = 2
yearfilterframe['relief'] = 'sunken'
ttk.Label(yearfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minyear=IntVar()
minyear.set(min_year)
ttk.Entry(yearfilterframe, textvariable=minyear).grid(column=1, row=0, sticky="E")
ttk.Label(yearfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxyear=IntVar()
maxyear.set(max_year)
ttk.Entry(yearfilterframe, textvariable=maxyear).grid(column=1, row=1, sticky="E")


numratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
numratingfilterframe.grid(column=0, row=1, sticky=(N, E))
numratingfilterframe['borderwidth'] = 2
numratingfilterframe['relief'] = 'sunken'
ttk.Label(numratingfilterframe, text="MIN").grid(column=0, row=0, sticky="E")
minratingcount=IntVar()
minratingcount.set(min_rating_count)
ttk.Entry(numratingfilterframe, textvariable=minratingcount).grid(column=1, row=0, sticky="W")
ttk.Label(numratingfilterframe, text="MAX").grid(column=0, row=1, sticky="E")
maxratingcount=IntVar()
maxratingcount.set(max_rating_count)
ttk.Entry(numratingfilterframe, textvariable=maxratingcount).grid(column=1, row=1, sticky="W")


meanratingfilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="# of Ratings")
meanratingfilterframe.grid(column=0, row=2, sticky=(N, E))
meanratingfilterframe['borderwidth'] = 2
meanratingfilterframe['relief'] = 'sunken'
ttk.Label(meanratingfilterframe, text="MIN").grid(column=0, row=0, sticky="W")
minrating=DoubleVar()
minrating.set(min_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=minrating).grid(column=1, row=0, sticky="E")
ttk.Label(meanratingfilterframe, text="MAX").grid(column=0, row=1, sticky="W")
maxrating=DoubleVar()
maxrating.set(max_mean_rating)
ttk.Entry(meanratingfilterframe, textvariable=maxrating).grid(column=1, row=1, sticky="E")

titlefilterframe = ttk.Labelframe(filterframe, padding="3 3 12 12", text="Title")
titlefilterframe.grid(column=0, row=3, sticky="nsew")
titlefilterframe['borderwidth'] = 2
titlefilterframe['relief'] = 'sunken'

searchphrase = StringVar()
ttk.Entry(titlefilterframe, textvariable=searchphrase).grid(column=0, row=0, sticky="nsew")
#ttk.Button(searchfilterframe, text="Search", command=search_filter).grid(column=1, row=0, sticky="nsew")

removed_iids = set()

def filter_data(*args):
    remove_iids = set()
    data = data_tree.get_children('')
    #print(data_tree.set(data[0]))    
    for iid in data:
        # Year
        if float(data_tree.set(iid)['Year']) < minyear.get():
            remove_iids.add(iid)
        if float(data_tree.set(iid)['Year']) > maxyear.get():
            remove_iids.add(iid)
        # Rating Count
        if float(data_tree.set(iid)['Rating Count']) < minratingcount.get():
            remove_iids.add(iid)
        if float(data_tree.set(iid)['Rating Count']) > maxratingcount.get():
            remove_iids.add(iid)
        # MEAN RATING
        if float(data_tree.set(iid)['Mean Rating']) < minrating.get():
            remove_iids.add(iid)
        if float(data_tree.set(iid)['Mean Rating']) > maxrating.get():
            remove_iids.add(iid)
        if data_tree.set(iid)['Title'].find(searchphrase.get()) == -1:
            remove_iids.add(iid)
    #data_tree.delete(list(remove_iids))
    for iid in remove_iids:
        data_tree.delete(iid)
    removed_iids.update(remove_iids)
    #TODO
    # 1. Update the other fields based on the new filtered data
    # 2. There is no way to go back in filtering right now

def reset_filters(*args):
    for iid in removed_iids:
        i = int(iid)
        item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating]
        data_tree.insert('', 'end', iid = str(i), values=item)
    sortby(data_tree, 'Title', False)
    removed_iids.clear()
    #TODO Reset the fields to their default values
    
def double_click(e):
    item = data_tree.selection()[0]
    #print("You clicked on", data_tree.set(item))
    #print(m_[int(item)])
    tl = Toplevel(root)
    #ttk.Label(tl, text=data_tree.set(item)).grid(column=0, row=0, sticky="E")
    m = m_[int(item)]
    
    def rate_movie(*args):
        print(m.title)
        print(rating.get())
        #ratings[m.title+str(m.year)]=rating.get()
        if rating.get() != "no-opinion":
            ratings[m] = rating.get()
        else:
            ratings.pop(m, None)
        tl.destroy()
    
    ttk.Label(tl, text=m.title + " (" + str(m.year) + ")", font=font.Font(family='Helvetica', size=14, weight='bold')).grid(column=0, row=0, sticky="W")
    if m.genres is not None:
        g = ", ".join(m.genres)
        ttk.Label(tl, text="Genres: " + g).grid(column=0, row=1, sticky="W")      
    if m.keywords is not None:
        #cnames = StringVar(value=countrynames)
        kl= Listbox(tl, listvariable=StringVar(value=m.keywords), height=5)
        kl.grid(column=0, row=2, sticky="nsew")
        vs = ttk.Scrollbar(tl, orient=VERTICAL, command=kl.yview)
        kl.configure(yscrollcommand=vs.set)
        vs.grid(column=1, row=2, sticky="nsew")
        hs = ttk.Scrollbar(tl, orient=HORIZONTAL, command=kl.xview)
        kl.configure(xscrollcommand=hs.set)
        hs.grid(column=0, row=3, sticky="nsew")
    rateframe=ttk.Frame(tl, padding="3 3 12 12")
    rateframe.grid(column=0, row=4, sticky="nsew")        
    rating = StringVar()
    if m in ratings:
        rating.set(ratings[m])
    else:
        rating.set("no-opinion")
    ttk.Radiobutton(rateframe, text='Like', variable=rating, value='like').grid(column=0, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='Dislike', variable=rating, value='dislike').grid(column=1, row=0, sticky="nsew")
    ttk.Radiobutton(rateframe, text='No opinion', variable=rating, value='no-opinion').grid(column=2, row=0, sticky="nsew")
    ttk.Button(rateframe, text="Submit", command=rate_movie).grid(column=1, row=1, sticky = "nsew")
        
    

buttonfilterframe = ttk.Frame(filterframe, padding="3 3 12 12")
buttonfilterframe.grid(column=0, row=4, sticky=(N, E))

ttk.Button(buttonfilterframe, text="Filter", command=filter_data).grid(column=0, row=0, sticky = "W")
ttk.Button(buttonfilterframe, text="Reset", command=reset_filters).grid(column=1, row=0, sticky = "E")

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=1900, to=2017 , showvalue=True, label="Minimum Year", command=filter_year)
#s.grid(column=0, row=0, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0, to=1e3, resolution=1, showvalue=True, label="Minimum # Ratings", command=filter_num_ratings)
#s.grid(column=0, row=1, sticky=W)

#s = Scale(filterframe, orient=HORIZONTAL, length=200, from_=0.0, to=10.0, resolution=0.1, showvalue=True, label="Minimum Mean Rating")
#s.grid(column=0, row=2, sticky=W)




dataframe = ttk.Labelframe(mainframe, padding="3 3 12 12", text="Data")
dataframe.grid(column=1, row=0, sticky="ne")
dataframe['borderwidth'] = 2
dataframe['relief'] = 'sunken'

searchfilterframe = ttk.Frame(dataframe, padding="3 3 12 12")
searchfilterframe['borderwidth'] = 2
searchfilterframe['relief'] = 'sunken'
searchfilterframe.grid(column=0, row=0, sticky="nsew")



# some of the following code is borrowed from https://www.daniweb.com/programming/software-development/threads/350266/creating-table-in-python

data_header = ['Title', 'Year', 'Rating Count', "Mean Rating"]
#data = [[m.title, m.year, m.num_ratings, m.mean_rating] for m in m_]

data_tree = ttk.Treeview(dataframe, columns=data_header, show="headings", selectmode='browse')

vsb = ttk.Scrollbar(dataframe, orient="vertical", command=data_tree.yview)
hsb = ttk.Scrollbar(dataframe, orient="horizontal", command=data_tree.xview)

data_tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
data_tree.grid(column=0, row=1, sticky='nsew')

data_tree.bind("<Double-1>", double_click)

vsb.grid(column=1, row=1, sticky='ns')
hsb.grid(column=0, row=2, sticky='ew')

dataframe.grid_columnconfigure(0, weight=1)
dataframe.grid_rowconfigure(0, weight=1)


for col in data_header:
    data_tree.heading(col, text=col.title(), command=lambda c=col: sortby(data_tree, c, 0))
    # adjust the column's width to the header string
    data_tree.column(col, width=tkFont.Font().measure(col.title()))
    
for i in range(len(m_)):
    item = [m_[i].title, m_[i].year, m_[i].num_ratings, m_[i].mean_rating]
    data_tree.insert('', 'end', iid = str(i), values=item)
    # adjust column's width if necessary to fit each value
    for ix, val in enumerate(item):
        col_w = tkFont.Font().measure(val)
        if data_tree.column(data_header[ix],width=None)<col_w:
            data_tree.column(data_header[ix], width=col_w)

root.mainloop()

In [48]:
ratings

{The Hunger Games	2012	0000012211	735760	7.2: 'like',
 No Country for Old Men	2007	0000001221	660480	8.1: 'like',
 The Lord of the Rings: The Two Towers	2002	0000000223	1177998	8.7: 'like',
 The Lion King	1994	0000001223	700578	8.5: 'like',
 Forrest Gump	1994	0000000233	1360088	8.8: 'like',
 One Flew Over the Cuckoo's Nest	1975	0000000232	725979	8.7: 'like',
 Requiem for a Dream	2000	0000001222	613783	8.4: 'like',
 Pirates of the Caribbean: The Curse of the Black Pearl	2003	0000001321	861332	8.0: 'like',
 Slumdog Millionaire	2008	0000001321	677178	8.0: 'like',
 The Shawshank Redemption	1994	0000000125	1812040	9.3: 'like',
 Fight Club	1999	0000000133	1451784	8.8: 'like',
 Pirates of the Caribbean: Dead Man's Chest	2006	0000012211	552373	7.3: 'like',
 Toy Story	1995	0000001322	674241	8.3: 'like',
 Inception	2010	0000000133	1584075	8.8: 'like',
 The Bourne Supremacy	2004	0000002311	372655	7.8: 'like',
 Back to the Future	1985	0000001222	792636	8.5: 'like',
 Braveheart	1995	0000001222	7880

In [49]:
pickle.dump(ratings, open( "ratings.p", "wb" ))

In [52]:
# Vectorize
# #Ratings, z-scored Mean Rating, #Ratings x z-scored meanrating, Genres, keywords
Genres = set()
Keywords = set()
for m in ratings:
    if m.genres is not None:
        for g in m.genres:
            Genres.add(g)
    if m.keywords is not None:
        for k in m.keywords:
            Keywords.add(k)

In [56]:
len(Genres)

17

In [55]:
len(Keywords)

5425

In [64]:
Genres = list(Genres)

In [66]:
Keywords = list(Keywords)

In [57]:
import numpy as np

In [58]:
r = []
for m in movies:
    r.append(m.mean_rating)

In [63]:
r_mean = np.mean(r)
r_var = np.var(r)

In [77]:
from scipy.sparse import csr_matrix, vstack

In [88]:
D = []
y = []
for m in m_:
    d = []
    d.append(m.num_ratings)
    d.append((m.mean_rating - r_mean)/r_var)
    d.append(d[0]*d[1])
    m_g = set()
    if m.genres is not None:
        m_g.update(m.genres)
    m_k = set()
    if m.keywords is not None:
        m_k.update(m.keywords)
    for g in Genres:
        if g in m_g:
            d.append(1)
        else:
            d.append(0)
    for k in Keywords: # VERY INEFFICIENT. FIX!
        if k in m_k:
            d.append(1)
        else:
            d.append(0)
    D.append(csr_matrix(d))
    if m in ratings:
        y.append(1)
    else:
        y.append(0)

In [89]:
len(D)

8000

In [90]:
d = D[0]

In [91]:
for d_ in D[1:]:
    d = vstack([d, d_])

In [92]:
d.shape

(8000, 5445)

In [93]:
len(y)

8000

In [94]:
y = np.array(y)

In [96]:
d[np.where(y==1)]

<45x5445 sparse matrix of type '<class 'numpy.float64'>'
	with 10032 stored elements in Compressed Sparse Row format>

In [97]:
from sklearn import svm

In [100]:
clf = svm.OneClassSVM(kernel='linear')

In [102]:
clf.fit(d[np.where(y==1)])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto',
      kernel='linear', max_iter=-1, nu=0.5, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

In [105]:
clf.coef_

<1x5445 sparse matrix of type '<class 'numpy.float64'>'
	with 2745 stored elements in Compressed Sparse Row format>

In [110]:
indices = np.argsort(np.abs(clf.coef_.toarray()[0]))

In [118]:
for i in indices[-10:]:
    print(clf.coef_.toarray()[0][i])

11.0
11.0
11.0
11.5
12.0
13.0
16.5
16.6460824028
10681331.9922
14352004.5


In [119]:
np.where(y==1)

(array([ 956, 1200, 1366, 1394, 1729, 2355, 2761, 2774, 2841, 3022, 3064,
        3523, 3588, 3737, 4235, 4501, 4711, 4820, 5016, 5018, 5264, 5476,
        5492, 5690, 6209, 6246, 6248, 6249, 6363, 6522, 6524, 6603, 6627,
        6778, 6804, 6805, 6806, 6858, 7139, 7149, 7154, 7460, 7462, 7610,
        7623], dtype=int64),)

In [124]:
np.where(clf.predict(d) != -1)

(array([ 105,  254, 1194, 1200, 1366, 1394, 1468, 1729, 2387, 2761, 2841,
        3022, 3066, 3523, 3537, 3564, 4235, 4384, 5018, 5131, 5454, 5476,
        5492, 5614, 5845, 5849, 6091, 6174, 6363, 6364, 6384, 6522, 6524,
        6562, 6804, 6805, 6806, 6858, 7006, 7139, 7149, 7271, 7323, 7623,
        7697], dtype=int64),)

In [140]:
df = (clf.decision_function(d)).flatten()

In [141]:
indices = np.argsort(df)

In [142]:
df [indices[:10]]

array([ -1.94573266e+13,  -1.93666824e+13,  -1.93286418e+13,
        -1.92817560e+13,  -1.91286447e+13,  -1.90857146e+13,
        -1.90761925e+13,  -1.89445972e+13,  -1.89306174e+13,
        -1.89170855e+13])

In [143]:
df[indices[-10:]]

array([  1.58551588e+13,   1.59138892e+13,   1.60498208e+13,
         1.63530164e+13,   1.69335970e+13,   1.92206447e+13,
         1.93267563e+13,   2.27793981e+13,   3.01119580e+13,
         3.35224193e+13])

In [144]:
indices[0]

3758

In [145]:
m_[3758]

Justin Bieber: Never Say Never	2011	8000000001	74557	1.6

In [146]:
indices[-1]

7139

In [147]:
m_[7139]

The Shawshank Redemption	1994	0000000125	1812040	9.3

In [150]:
for i in indices[:10]:
    print(m_[i])

Justin Bieber: Never Say Never	2011	8000000001	74557	1.6
Disaster Movie	2008	6100000000	77218	1.9
Epic Movie	2007	4110000000	92341	2.3
Reis	2017	7000000001	69057	1.9
Meet the Spartans	2008	4110000000	91599	2.7
Battlefield Earth	2000	4100000000	67335	2.4
Gunday	2014	8000000000	54884	2.1
Son of the Mask	2005	4110000000	43255	2.2
Dragonball Evolution	2009	4110000000	59524	2.7
The Hottie & the Nottie	2008	6000000000	34014	1.9


In [151]:
for i in indices[-10:]:
    print(m_[i])

"Game of Thrones"	2011	0000000016	1154888	9.5
The Lord of the Rings: The Fellowship of the Ring	2001	0000000123	1321017	8.8
The Lord of the Rings: The Return of the King	2003	0000000124	1299875	8.9
The Godfather	1972	0000000125	1237499	9.2
Forrest Gump	1994	0000000233	1360088	8.8
Pulp Fiction	1994	0000000123	1418954	8.9
Fight Club	1999	0000000133	1451784	8.8
Inception	2010	0000000133	1584075	8.8
The Dark Knight	2008	0000000124	1792361	9.0
The Shawshank Redemption	1994	0000000125	1812040	9.3
